# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [1]:
# Import necessary packages
import sqlite3 as sq3



In [2]:
# Create the database school.sqlite 
conn = sq3.connect('school')

## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table. 

In [3]:
# Your code here
cur = conn.cursor()


In [4]:
cur.execute('''DROP TABLE contactInfo''')

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur. fetchall())

[('contact_info',), ('grades',)]


In [5]:
cur.execute('''
        CREATE TABLE IF NOT EXISTS contactInfo(
            id INTEGER PRIMARY KEY,
            firstName TEXT,
            lastName TEXT,
            role TEXT,
            telephone INTEGER,
            street TEXT,
            city TEXT,
            state TEXT,
            zipcode TEXT)

''')

## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [6]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur. fetchall())

[('contact_info',), ('grades',), ('contactInfo',)]


In [7]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

In [8]:
contacts

[{'firstName': 'Christine',
  'lastName': 'Holden',
  'role': 'staff',
  'telephone ': 2035687697,
  'street': '1672 Whitman Court',
  'city': 'Stamford',
  'state': 'CT',
  'zipcode ': '06995'},
 {'firstName': 'Christopher',
  'lastName': 'Warren',
  'role': 'student',
  'telephone ': 2175150957,
  'street': '1935 University Hill Road',
  'city': 'Champaign',
  'state': 'IL',
  'zipcode ': '61938'},
 {'firstName': 'Linda',
  'lastName': 'Jacobson',
  'role': 'staff',
  'telephone ': 4049446441,
  'street': '479 Musgrave Street',
  'city': 'Atlanta',
  'state': 'GA',
  'zipcode ': '30303'},
 {'firstName': 'Andrew',
  'lastName': 'Stepp',
  'role': 'student',
  'telephone ': 7866419252,
  'street': '2981 Lamberts Branch Road',
  'city': 'Hialeah',
  'state': 'Fl',
  'zipcode ': '33012'},
 {'firstName': 'Jane',
  'lastName': 'Evans',
  'role': 'student',
  'telephone ': 3259909290,
  'street': '1461 Briarhill Lane',
  'city': 'Abilene',
  'state': 'TX',
  'zipcode ': '79602'},
 {'firstNa

In [9]:
# Iterate over the contact list and populate the contactInfo table here
for person in contacts:
    cur.execute('''
            INSERT INTO contactInfo VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
    ''', [None, 
          person['firstName'], 
          person['lastName'], 
          person['role'], 
          person['telephone '], 
          person['street'], 
          person['city'], 
          person['state'], 
          person['zipcode ']])
conn.commit()

**Query the Table to Ensure it is populated**

In [10]:
# Your code here 
cur.execute('''SELECT firstName FROM contactInfo ''').fetchall()

[('Christine',),
 ('Christopher',),
 ('Linda',),
 ('Andrew',),
 ('Jane',),
 ('Jane',),
 ('Mary',),
 ('Ed',)]

## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [11]:
# Your code here

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [12]:
# Create the grades table
cur.execute('''
        CREATE TABLE IF NOT EXISTS grades(
            userId INTEGER NOT NULL,
            courseId INTEGER NOT NULL,
            grade INTEGERNOT NULL,
            PRIMARY KEY(userId, courseId, grade))

''')

## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [13]:
# Find the duplicate entry
cur.execute('''
        SELECT a.*
        FROM contactInfo a
        JOIN (SELECT firstName, telephone, COUNT(*)
        FROM contactInfo 
        GROUP BY firstName, telephone
        HAVING count(*) > 1 ) b
        ON a.firstName = b.firstName
        AND a.telephone = b.telephone
        ORDER BY a.firstName
''').fetchall()

[(5,
  'Jane',
  'Evans',
  'student',
  3259909290,
  '1461 Briarhill Lane',
  'Abilene',
  'TX',
  '79602'),
 (6,
  'Jane',
  'Evans',
  'student',
  3259909290,
  '1461 Briarhill Lane',
  'Abilene',
  'TX',
  '79602')]

In [14]:
# Delete the duplicate entry
cur.execute('''
        DELETE FROM contactInfo
        
        WHERE id=6
''')

In [15]:
# Check that the duplicate entry was removed
cur.execute('''SELECT firstName, telephone FROM contactInfo ''').fetchall()

[('Christine', 2035687697),
 ('Christopher', 2175150957),
 ('Linda', 4049446441),
 ('Andrew', 7866419252),
 ('Jane', 3259909290),
 ('Mary', 9075772295),
 ('Ed', 5179695576)]

## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [18]:
# Update Ed's address
cur.execute('''
        UPDATE contactInfo
        SET street="2910 Simpson Avenue",
            city="York",
            state="PA",
            zipcode="17403"
        WHERE firstName="Ed"
''').fetchall()

[]

In [19]:
# Query the database to ensure the change was made
cur.execute('''SELECT firstName, street, city, state, zipcode FROM contactInfo ''').fetchall()

[('Christine', '1672 Whitman Court', 'Stamford', 'CT', '06995'),
 ('Christopher', '1935 University Hill Road', 'Champaign', 'IL', '61938'),
 ('Linda', '479 Musgrave Street', 'Atlanta', 'GA', '30303'),
 ('Andrew', '2981 Lamberts Branch Road', 'Hialeah', 'Fl', '33012'),
 ('Jane', '1461 Briarhill Lane', 'Abilene', 'TX', '79602'),
 ('Mary', '3975 Jerry Toth Drive', 'Ninilchik', 'AK', '99639'),
 ('Ed', '2910 Simpson Avenue', 'York', 'PA', '17403')]

## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [20]:
# Your code here
conn.commit()

conn.close()

## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 